# 스킨케어 순위 csv로 저장하기

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 타겟 URL
url = "https://www.oliveyoung.co.kr/store/main/getBestList.do?dispCatNo=900000100100001&fltDispCatNo=10000010001&pageIdx=1&rowsPerPage=100"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://www.oliveyoung.co.kr/"
}

res = requests.get(url, headers=headers)
soup = BeautifulSoup(res.text, 'html.parser')
items = soup.select("ul.cate_prd_list > li")

products = []
for idx, item in enumerate(items, start=1):  # 순위: 1부터 시작
    name_tag = item.select_one("div.prd_name a")
    price_nums = item.select("p.prd_price span.tx_num")

    name = name_tag.text.strip() if name_tag else None

    # 가격 처리
    if len(price_nums) > 1:
        original_price = price_nums[0].text.strip()
        discounted_price = price_nums[1].text.strip()
    elif len(price_nums) == 1:
        original_price = ""
        discounted_price = price_nums[0].text.strip()
    else:
        original_price = ""
        discounted_price = ""

    if name:
        products.append({
            "순위": idx,
            "상품명": name,
            "정가": original_price,
            "할인가": discounted_price
        })

# DataFrame 저장
df = pd.DataFrame(products)
df.to_csv("skincare_ranking1.csv", index=False, encoding="utf-8-sig")
print("데이터 저장 완료! 총", len(df), "개")


데이터 저장 완료! 총 100 개


# mysql에 넣기

## SQLite

In [38]:
import sqlite3

# SQLite 연결 및 DB 파일 생성 (없으면 자동 생성)
conn = sqlite3.connect("oliveyoung.db")
cursor = conn.cursor()

# 테이블 생성 (없으면 생성)
cursor.execute("""
CREATE TABLE IF NOT EXISTS oliveyoung_skincare (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    rank_order INTEGER,
    name TEXT NOT NULL,
    original_price TEXT,
    discounted_price TEXT
)
""")

# 데이터 삽입
insert_query = """
INSERT INTO oliveyoung_skincare (rank_order, name, original_price, discounted_price)
VALUES (?, ?, ?, ?)
"""

for _, row in df.iterrows():
    values = (
        int(row["순위"]),
        row["상품명"],
        row["정가"],
        row["할인가"]
    )
    cursor.execute(insert_query, values)

# 커밋 및 연결 종료
conn.commit()
conn.close()

print("SQLite 저장 완료: oliveyoung.db, 총", len(df), "개")


SQLite 저장 완료: oliveyoung.db, 총 100 개


## MySQL에 연결 후 저장

In [ ]:
import pymysql

# MySQL 연결 설정
conn = pymysql.connect(
    host='localhost',       
    user='root',            
    password='1234',    
    database='oliveyoung',  # 위에서 만든 DB 이름
    charset='utf8mb4',
    autocommit=True
)

cursor = conn.cursor()

# 테이블 생성 (없으면 생성)
cursor.execute("""
CREATE TABLE IF NOT EXISTS oliveyoung_skincare (
    id INT AUTO_INCREMENT PRIMARY KEY,
    rank_order INT,
    name VARCHAR(255) NOT NULL,
    original_price VARCHAR(50),
    discounted_price VARCHAR(50)
)
""")

# 데이터 삽입
insert_query = """
INSERT INTO oliveyoung_skincare (rank_order, name, original_price, discounted_price)
VALUES (%s, %s, %s, %s)
"""

for _, row in df.iterrows():
    values = (
        int(row["순위"]),
        row["상품명"],
        row["정가"],
        row["할인가"]
    )
    cursor.execute(insert_query, values)

# 종료
cursor.close()
conn.close()

print("MySQL DB 저장 완료! 총", len(df), "개")


MySQL DB 저장 완료! 총 100 개
